In [48]:
import pandas as pd

In [49]:
df1 = pd.read_excel("active_listening_activity.xlsx")
df2 = pd.read_excel("math_activity.xlsx")
df3 = pd.read_excel("speaking_activity.xlsx")
df4 = pd.read_excel("writing_activity.xlsx")
df5 = pd.read_excel("reading_comprehension_activity.xlsx")
df6 = pd.read_excel("science_activity.xlsx")

In [50]:
df1.head()

,Unnamed: 0,content,detail
0,0,making decisions and solving problems,— analyzing information and evaluating results...
1,1,getting information,"— observing, receiving, and otherwise obtainin..."
2,2,performing for or working directly with the pu...,— performing for people or dealing directly wi...
3,3,evaluating information to determine compliance...,— using relevant information and individual ju...
4,4,processing information,"— compiling, coding, categorizing, calculating..."


In [51]:
full_list = []
for i in [df1,df2,df3,df4,df5,df6]:
    for content, detail in zip(i.content,i.detail):
        full_list.append(content+detail)
full_list[:10]

['making decisions and solving problems— analyzing information and evaluating results to choose the best solution and solve problems',
 'getting information— observing, receiving, and otherwise obtaining information from all relevant sources',
 'performing for or working directly with the public— performing for people or dealing directly with the public. this includes serving customers in restaurants and stores, and receiving clients or guests',
 'evaluating information to determine compliance with standards— using relevant information and individual judgment to determine whether events or processes comply with laws, regulations, or standards',
 'processing information— compiling, coding, categorizing, calculating, tabulating, auditing, or verifying information or data',
 'updating and using relevant knowledge— keeping up-to-date technically and applying new knowledge to your job',
 'judging the qualities of objects, services, or people— assessing the value, importance, or quality of t

In [52]:
work_content_list_unrep = list(set(full_list))
len(work_content_list_unrep)

41

In [53]:
unrep_content_list = []
unrep_detail_list = []
for i in work_content_list_unrep:
    unrep_content_list.append(i.split("— ")[0])
    unrep_detail_list.append(i.split("— ")[1])

content_based_df = pd.DataFrame({"content":unrep_content_list,"detail":unrep_detail_list})
content_based_df.to_excel("working content.xlsx")

In [54]:
from nltk.corpus import stopwords
import re

# split the sentences and store in the list (can be used for deducting the classification of words)
sentences = unrep_detail_list
# store cleaned sentences
cleaned_work_exp_0 = []
# clean data
for i in range(len(sentences)):
    review = sentences[i].replace("etc.","").replace("e.g.","").replace("i.e.","")
    # only keep letters, + and - in the original passage
    review = re.sub('[^a-zA-Z+-]'," ",review)
    review = review.replace(" one "," ").replace(" two "," ").replace(" three "," ").replace(" four "," ").replace(" five "," ").replace(" six "," ").replace(" seven "," ").replace(" eight "," ").replace(" nine "," ").replace(" ten "," ")
    review = review.lower()
    review = review.split()
    # delete stop words and transfer words to unified format
    review = [word for word in review if not word in set(stopwords.words("english"))]
    review = " ".join(review)
    cleaned_work_exp_0.append(review)


In [65]:
import spacy

# English pipelines include a rule-based lemmatizer
nlp = spacy.load("en_core_web_sm")
lemmatizer = nlp.get_pipe("lemmatizer")

cleaned_work_exp = []
for i in cleaned_work_exp_0:
    doc = nlp(i)
    cleaned_work_exp.append(" ".join([token.lemma_ for token in doc]))

cleaned_work_exp[:10]

['compile coding categorize calculate tabulate auditing verifying information datum',
 'use relevant information individual judgment determine whether event process comply law regulation standard',
 'translate explain information mean use',
 'run maneuvering navigate driving vehicle mechanized equipment forklift passenger vehicle aircraft watercraft',
 'service repair calibrate regulate fine - tuning testing machine device equipment operate primarily basis electrical electronic mechanical principle',
 'identify educational need other develop formal educational training program class teach instruct other',
 'provide guidance expert advice management group technical systems- process - relate topic',
 'provide documentation detailed instruction drawing specification tell other device part equipment structure fabricate construct assemble modify maintain use',
 'establish long - range objective specify strategy action achieve',
 'use hand arm handle instal positioning move material manipula

In [56]:
signal = {}
flag = 0
for i in cleaned_work_exp:
    signal[i] = flag
    flag = flag+1

In [57]:
import json
with open("/Users/zhiyuzhang/Desktop/ocbang/Matching System/O*NET web scraping/preliminary_attempt/clean_data/candidate_profile.json","r") as f:
    data = json.load(f)

In [58]:
# just a try
try_data = data["10000"]["working_experiences"][:15]

In [59]:
try_data

['marketing contractor responsibilitie global adsense - small - medium publisher marketing lead global marketing strategy line adsense business product priority drive acquisition growth',
 'act subject matter expert adsense marketing collaborating multiple stakeholder write narrative provide strategic guidance channel owner website blog email social medium youtube seo pay',
 'work closely product sale go - to - market team align consult wide commercialisation plan',
 'lead research drive content development improve pay organic acquisition campaign',
 'increase acquisition conversion - month pay medium beta campaign update creative improving target',
 'manage marketing budget - instal process rigor improve roi reporting spend efficiency',
 'central approver adsense global marcomms ensure google brand continuity consistency throughout',
 'lead think leadership initiative sale stakeholder drive centralised cross - channel approach - focus proactive engage helpful simplified publisher comm

In [60]:
flag = 0
for i in try_data:
    print(flag)
    for single_word in i.split():
        for n in list(signal.keys()):
            for h in n.split():
                if single_word == h and single_word != "-":
                    print(single_word,n)
    flag = flag+1


0
strategy establish long - range objective specify strategy action achieve
product develop designing create new application idea relationship system product include artistic contribution
1
expert provide guidance expert advice management group technical systems- process - relate topic
write use computer computer system include hardware software program write software set function enter data process information
write communicate people outside organization represent organization customer public government external source information exchange person write telephone e - mail
write provide information supervisor co - worker subordinate telephone write form e - mail person
write enter transcribe recording store maintain information write electronic magnetic form
provide provide guidance expert advice management group technical systems- process - relate topic
provide provide documentation detailed instruction drawing specification tell other device part equipment structure fabricate constru

In [69]:
# recognize useful words in target matrix
doc = nlp('establish long - range objective specify strategy action achieve')
for token in doc:
    print("{}-{}".format(token,token.pos_))

# retain NOUN,VERB, (ADJ)-(need?), 

establish-VERB
long-ADJ
--PUNCT
range-NOUN
objective-ADJ
specify-NOUN
strategy-NOUN
action-NOUN
achieve-VERB


In [73]:
# actually can be manually identified to increase the accuracy
cleaned_model = []
for i in cleaned_work_exp:
    doc = nlp(i)
    temp_list = []
    for token in doc:
        if token.pos_ in ("VERB","ADJ","NOUN") and token.orth_ != "other" and token.orth_ !="detailed":
            temp_list.append(token.orth_)
        else:
            continue
    cleaned_sentence = " ".join(temp_list)
    cleaned_model.append(cleaned_sentence)

cleaned_model

['compile coding categorize calculate tabulate auditing verifying information datum',
 'use relevant information individual judgment determine event process comply law regulation standard',
 'translate explain information mean use',
 'run maneuvering navigate driving vehicle mechanized equipment forklift passenger vehicle aircraft watercraft',
 'service repair calibrate regulate tuning testing machine device equipment operate basis electrical electronic mechanical principle',
 'identify educational need develop formal educational training program class teach instruct',
 'provide guidance expert advice management group technical systems- process relate topic',
 'provide documentation instruction drawing specification tell device part equipment structure fabricate construct assemble modify maintain use',
 'establish long range objective specify strategy action achieve',
 'use hand arm handle instal positioning move material manipulate thing',
 'get member group work accomplish task',
 'u

In [74]:
final_activity = pd.DataFrame({"Category":unrep_content_list,"KeyWords":cleaned_model})
final_activity.to_excel("pre_softskills_matrix.xlsx")